In [18]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import requests
import json

In [19]:
df_UBC = pd.read_csv("Gbif_UBC.csv", delimiter="\t", low_memory=False)
df_UBC = df_UBC.head(200)

In [20]:
geometry = [Point(xy) for xy in zip (df_UBC['decimalLongitude'], df_UBC['decimalLatitude'])]

In [32]:
k = df_UBC['datasetKey'].unique()
n = ([])
for key in k:
    url = "http://api.gbif.org/v1/dataset/" + key
    name = requests.get(url).json()['title']
    n.append(name)
conv = dict(zip(k,n))  
print(conv)

{'50c9509d-22c7-4a22-a47d-8c48425ef4a7': u'iNaturalist Research-grade Observations', '2429287b-ef65-4cfd-afcc-11cc3ba95cca': u'University of Alberta Vascular Plant Herbarium (ALTA-VP)', '07fd0d79-4883-435f-bba1-58fef110cd13': u'University of British Columbia Herbarium (UBC) - Vascular Plant Collection', '4fa7b334-ce0d-4e88-aaae-2e0c138d049e': u'EOD - eBird Observation Dataset', 'b1047888-ae52-4179-9dd5-5448ea342a24': u'Xeno-canto - Bird sounds from around the world', '4edd9396-59df-4b01-9e29-dc21a59f9963': u'University of British Columbia Herbarium (UBC) - Bryophytes Collection', '27b4ff4b-29c3-4017-9c48-3750861392f7': u'University of North Carolina Chapel Hill Herbarium', 'ad43e954-dd79-4986-ae34-9ccdbd8bf568': u'Geographically tagged INSDC sequences'}


In [33]:
df_UBC['datasetName'] = df_UBC['datasetKey']
df_UBC.replace({'datasetName': conv})

,gbifID,datasetKey,occurrenceID,kingdom,phylum,class,order,family,genus,species,...,dateIdentified,license,rightsHolder,recordedBy,typeStatus,establishmentMeans,lastInterpreted,mediaType,issue,datasetName
0,2244417231,2429287b-ef65-4cfd-afcc-11cc3ba95cca,urn:catalog:ALTA-VP:140446,Plantae,Tracheophyta,Pinopsida,Pinales,Pinaceae,Pinus,Pinus coulteri,...,1959-01-01T00:00:00Z,CC0_1_0,University of Alberta Museums,"Krajina, Vladimir J.",NaN,NaN,2019-05-28T18:11:41.642Z,NaN,NaN,University of Alberta Vascular Plant Herbarium...
1,2244417216,2429287b-ef65-4cfd-afcc-11cc3ba95cca,urn:catalog:ALTA-VP:140449,Plantae,Tracheophyta,Liliopsida,Liliales,Liliaceae,Prosartes,Prosartes hookeri,...,1950-01-01T00:00:00Z,CC0_1_0,University of Alberta Museums,"Krajina, Vladimir J.",NaN,NaN,2019-05-28T18:11:41.452Z,NaN,NaN,University of Alberta Vascular Plant Herbarium...
2,2244417175,2429287b-ef65-4cfd-afcc-11cc3ba95cca,urn:catalog:ALTA-VP:140526,Plantae,Tracheophyta,Magnoliopsida,Saxifragales,Saxifragaceae,Tellima,Tellima grandiflora,...,NaN,CC0_1_0,University of Alberta Museums,"Krajina, Vladimir J.",NaN,NaN,2019-05-28T18:11:41.577Z,NaN,NaN,University of Alberta Vascular Plant Herbarium...
3,2244417140,2429287b-ef65-4cfd-afcc-11cc3ba95cca,urn:catalog:ALTA-VP:140455,Plantae,Tracheophyta,Magnoliopsida,Sapindales,Sapindaceae,Acer,Acer palmatum,...,2011-01-01T00:00:00Z,CC0_1_0,University of Alberta Museums,"Shcherbinina, Anna",NaN,NaN,2019-05-28T18:11:41.494Z,NaN,NaN,University of Alberta Vascular Plant Herbarium...
4,2244417133,2429287b-ef65-4cfd-afcc-11cc3ba95cca,urn:catalog:ALTA-VP:140456,Plantae,Tracheophyta,Magnoliopsida,Asterales,Asteraceae,Anaphalis,Anaphalis margaritacea,...,1959-01-01T00:00:00Z,CC0_1_0,University of Alberta Museums,"Lesko, G.",NaN,NaN,2019-05-28T18:11:41.405Z,NaN,NaN,University of Alberta Vascular Plant Herbarium...
5,2244330798,50c9509d-22c7-4a22-a47d-8c48425ef4a7,https://www.inaturalist.org/observations/25636793,Plantae,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Vicia,Vicia sativa,...,2019-05-23T20:29:42Z,CC_BY_NC_4_0,Laura,Laura,NaN,NaN,2019-05-27T11:45:49.459Z,STILLIMAGE,GEODETIC_DATUM_ASSUMED_WGS84;COORDINATE_ROUNDED,iNaturalist Research-grade Observations
6,2244225693,50c9509d-22c7-4a22-a47d-8c48425ef4a7,https://www.inaturalist.org/observations/25249531,Animalia,Chordata,Aves,Passeriformes,Muscicapidae,Turdus,Turdus migratorius,...,2019-05-17T04:21:45Z,CC_BY_NC_4_0,mefisher,mefisher,NaN,NaN,2019-05-27T11:44:08.500Z,NaN,GEODETIC_DATUM_ASSUMED_WGS84;COORDINATE_ROUNDED,iNaturalist Research-grade Observations
7,2244223606,50c9509d-22c7-4a22-a47d-8c48425ef4a7,https://www.inaturalist.org/observations/25229414,Plantae,Tracheophyta,Magnoliopsida,Rosales,Rosaceae,Rubus,Rubus parviflorus,...,2019-05-16T19:58:07Z,CC_BY_NC_4_0,Emma Harrower,Emma Harrower,NaN,NaN,2019-05-27T11:44:05.568Z,STILLIMAGE,GEODETIC_DATUM_ASSUMED_WGS84;COORDINATE_ROUNDED,iNaturalist Research-grade Observations
8,2244221995,50c9509d-22c7-4a22-a47d-8c48425ef4a7,https://www.inaturalist.org/observations/25161662,Plantae,Tracheophyta,Magnoliopsida,Fabales,Fabaceae,Trifolium,Trifolium dubium,...,2019-05-15T15:09:39Z,CC_BY_NC_4_0,jamie_fenneman,jamie_fenneman,NaN,NaN,2019-05-27T11:43:58.324Z,STILLIMAGE,GEODETIC_DATUM_ASSUMED_WGS84;COORDINATE_ROUNDED,iNaturalist Research-grade Observations
9,2244206787,50c9509d-22c7-4a22-a47d-8c48425ef4a7,https://www.inaturalist.org/observations/21339520,Animalia,Chordata,Aves,Apodiformes,Trochilidae,Calypte,Calypte anna,...,2019-03-17T23:49:10Z,CC_BY_NC_4_0,Kyle Blaney,Kyle Blaney,NaN,NaN,2019-05-27T11:37:54.235Z,NaN,GEODETIC_DATUM_ASSUMED_WGS84,iNaturalist Research-grade Observations


In [34]:
geo_df = gpd.GeoDataFrame(df_UBC, geometry = geometry, crs = {'init': 'epsg:4326'})

In [35]:
geo_df.to_file("leaflet/UBC_alt.geojson", driver="GeoJSON")